In [ ]:
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
df = pd.DataFrame()

for file_name in os.listdir():
    if file_name.endswith('.xlsx'):
        airline_df = pd.read_excel(file_name)

        df = pd.concat([df, airline_df], ignore_index=True)

df = df.iloc[:,5:13].dropna().reset_index(drop=True)

In [ ]:
df['Recommended'] = df['Recommended'].apply(lambda x: 0 if x == 'no' else 1)
df = df.astype(int)

In [ ]:
recommendation_n = df['Recommended'].value_counts()

plt.figure(figsize=(12,8))
plt.bar(['No', 'Yes'], recommendation_n, color=['red', 'blue'])
plt.ylabel('# of Customers')
plt.title('Recommended?')
plt.show()

In [ ]:
df['Recommended'].value_counts()

In [ ]:
df_yes = df[df['Recommended'] == 1]
df_no = df[df['Recommended'] == 0]

df_yes_sampled = df_yes.sample(n=len(df_no), replace=False, random_state=34)

df_balanced = pd.concat([df_yes_sampled, df_no], axis=0).reset_index(drop=True)

In [ ]:
df_balanced['Recommended'].value_counts()

In [ ]:
X = df.drop('Recommended', axis=1)
y = df['Recommended']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=34
)

model = LogisticRegression()
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=['Not Recommended', 'Recommended'],
    yticklabels=['Not Recommended', 'Recommended']
)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
Index(['Seat Comfort', 'Cabin Staff Service', 'Food & Beverages',
       'Inflight Entertainment', 'Ground Service', 'Wifi & Connectivity',
       'Value For Money', 'Recommended'],
      dtype='object')

In [ ]:
def predict_recommendation(model):

    seat_comfort = int(input('Seat Comfort: '))
    cabin_staff_service = int(input('Cabin Staff Service: '))
    food_beverages = int(input('Food & Beverages: '))
    inflight_entertainment = int(input('Inflight Entertainment: '))
    ground_service = int(input('Ground Service: '))
    wifi_connectivity = int(input('Wifi & Connectivity: '))
    value_for_money = int(input('Value For Money: '))

    custom_data = {
        'Seat Comfort': [seat_comfort],
        'Cabin Staff Service': [cabin_staff_service],
        'Food & Beverages': [food_beverages],
        'Inflight Entertainment': [inflight_entertainment],
        'Ground Service': [ground_service],
        'Wifi & Connectivity': [wifi_connectivity],
        'Value For Money': [value_for_money]
    }
    custom_df = pd.DataFrame(custom_data)

    prediction = model.predict(custom_df)

    return prediction[0]

result = predict_recommendation(model)

if result == 1:
    print("Customer recommends.")
else:
    print("Customer does not recommend.")